In [11]:
import numpy as np
import os
import configparser
import copy
from collections import defaultdict

#获取数据
def get_data(filename):
    lines = open(filename,'r').readlines()
    items = []
    for l in lines:
        tmp = l.strip() #删除字符串头尾空白字符
        item = [i for i in tmp.split(",")] #分割元素转成数组 >>[0, 4828, 774, 207, 7460, 7465, 3768, 10221, 22435]
        items.append(item)

    return items


class Site:
    def __init__(self):
        self.bandwidth = 0
        self.overload = False #1超载
        self.free = 0 #剩余带宽

class User:
    def __init__(self):
        self.candidate_sites = []
        self.n = 0

    #计算候选节点的平均带宽
    def Avg(self,ssites,sites_head):

        s = [ssites[1][sites_head.index(v)] for v in self.candidate_sites]
        avg = int(sum(s)/self.n)
        return avg

    def ScaleAndSum(self,s,s_head):
        # print(self.candidate_sites)
        s_site = [s[1][s_head.index(v)] for v in self.candidate_sites]

        s_sum = sum(s_site)
        ss = [v/s_sum for v in s_site]

        return ss,s_sum

In [13]:
Demand = get_data("../data/demand1.csv")
Qos = get_data("../data/qos1.csv")
Site_bandwidth = get_data("../data/site_bandwidth1.csv")

# os.makedirs("../output")
# data = open("../output/solution.txt", 'w', encoding="utf-8")

config = configparser.ConfigParser()
config.read("../data/config.ini")
qos_constraint = int(config.get("config","qos_constraint"))

#用户需求
demand_head = Demand[0][1:]
demand_value = [[int(v) for v in i[1:]] for i in Demand[1:]]


#ssites[1] = bandwidth,ssites[2] = free, ssite[3] = overload
sites2index_dict = dict() #节点索引
index2isites_dict = dict()
ssites = np.zeros((3,len(Site_bandwidth)-1),dtype=int)
sites_head = []
for i,s in enumerate(Site_bandwidth[1:]):
    sites_head.append(s[0])
    # ssites[0][i] = i
    ssites[0][i] = s[1]
    ssites[1][i] = s[1]
    index2isites_dict[i] = s[0]
    sites2index_dict[s[0]] = i

print(ssites,sites2index_dict)


[[5088 6191 7294 8397 9500 3110 3610 3410 1000 1010 1020 1030 1040 1050
  1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190
  1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330
  1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470
  1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610
  1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750
  1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890
  1900 1910]
 [5088 6191 7294 8397 9500 3110 3610 3410 1000 1010 1020 1030 1040 1050
  1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190
  1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330
  1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470
  1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610
  1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750
  1760 1770 1780 1790 1800 1810 1820 1830 1840 1850

In [15]:

#初始化user
users = dict()

qos_head = Qos[0][1:]

for u in qos_head:
    users[u] = User()

for item in Qos[1:]:
    s = item[0] #节点名
    for index,u in enumerate(item[1:]):
        if int(u) < qos_constraint: #筛选符合user条件的节点
            users[qos_head[index]].n += 1
            users[qos_head[index]].candidate_sites.append(s)


In [22]:
output = []
for demand in demand_value:
    out_user = []

    users_tmp = copy.deepcopy(users)
    sites_tmp = ssites.copy()
    for time_index, u in enumerate(users_tmp):
        user_demand = demand[demand_head.index(u)]
        if user_demand == 0:
            # print("%s:"%(u))
            out_user.append([u])
            continue


        sites_scales,free_sum = users_tmp[u].ScaleAndSum(sites_tmp, sites_head)
        # print(sites_scales)


        out = defaultdict(int)
        user_demand_1 = user_demand

        #
        if free_sum <= user_demand:
            #全部分配
            for index, s in enumerate(users_tmp[u].candidate_sites):
                    s_i = sites_head.index(s) #site索引
                    if sites_tmp[2][s_i] : #是否超载
                        continue

                    out[s] += sites_tmp[1][s_i]
                    user_demand_1 -= sites_tmp[1][s_i]
                    sites_tmp[1][s_i] = 0
                    sites_tmp[2][s_i] = 1 #超载
        else:
            while user_demand_1 != 0:
                # print("u:{}".format(user_demand_1))
                while user_demand_1 > 100:
                    # print("user demand:{}".format(user_demand_1),end=" ")
                    sites_scales,free_sum = users_tmp[u].ScaleAndSum(sites_tmp, sites_head)
                    # print("free_sum:{}-{}".format(free_sum,sites_scales))
                    sites_scale = [int(user_demand_1*r) for r in sites_scales] #候选节点按比例分配的带宽
                    scale_sum = sum(sites_scale)
                    # print("scale_sum:{}{}".format(scale_sum,sites_scale),end=' ')

                    #每个节点按比例分配给user demand
                    for index, s in enumerate(users_tmp[u].candidate_sites):
                        s_i = sites_head.index(s) #site索引
                        if sites_tmp[2][s_i]: #是否超载
                            continue
                        #按候选节点顺序更新空余带宽
                        sites_tmp[1][s_i] -= sites_scale[index] #free 减去对应分配比例的带宽
                        user_demand_1 -= sites_scale[index]     #demand更新
                        if sites_tmp[1][s_i] == 0:              #当free恰好为0时超载，节点带宽极小时发生，可优化
                            sites_tmp[2][s_i] == 1 #超载

                        # out.append((s,sites_scale[index]))
                        out[s] += sites_scale[index]            #记录user_i当前site_j按比例分配的带宽


                #处理边缘情况，当user_demand小于100时，按比例乘得的数可能会很小，如0.01 * 100 当前节点分配1带宽，若继续按比例分配会进入循环
                #按节点顺序分配，如果剩余带宽不够，则全部分配
                n = 0
                for index, s in enumerate(users_tmp[u].candidate_sites):
                    s_i = sites_head.index(s) #site索引
                    if sites_tmp[2][s_i] : #是否超载
                        n += 1
                        continue

                    #按候选节点顺序更新空余带宽/可以加排序（待优化）/若当前节点带宽不够，则全部分配给user并记录超载
                    if sites_tmp[1][s_i] <= user_demand_1:
                        # out.append((s,sites_tmp[1][s_i]))
                        out[s] += sites_tmp[1][s_i]
                        user_demand_1 -= sites_tmp[1][s_i]
                        sites_tmp[1][s_i] = 0
                        sites_tmp[2][s_i] = 1 #超载
                    else:
                        # out.append((s,sites_tmp[1][s_i]))
                        out[s] += user_demand_1
                        sites_tmp[1][s_i] -= user_demand_1
                        user_demand_1 = 0

                if n == users_tmp[u].n:
                    break





        out_user.append([u])
        out_user[time_index].append(out)
        # break

    output.append(out_user)
    # print(sites_tmp)
    # break

In [23]:
for o in output:
    # print("---")
    for outuser in o:
       if len(outuser) == 1:
           print("{}:".format(outuser[0]))
       else:
            print("{}:".format(outuser[0]),end='')
            n = 1
            for k,v in outuser[1].items():
                if n == len(outuser[1]):
                    # print("<{},{}>".format(k,v),file=data)
                    print("<{},{}>".format(k,v))
                else:
                    # print("<{},{}>".format(k, v), end=",", file=data)
                    print("<{},{}>".format(k, v), end=",")
                    n += 1
    # print(sites_tmp)
    # break








---
A:<A,5088>,<B,6191>,<C,7294>,<D,8397>,<E,9500>
B:<Dn,2029>,<P,2347>,<Q,2217>,<AD,721>,<AF,728>,<AG,734>,<Ab,767>,<CS,1027>,<CT,1034>,<DU,1183>,<DV,1190>,<Dw,1203>
C:<AJ,247>,<Do,235>,<AS,237>,<AV,239>,<Ab,84>,<Dq,245>,<Ag,247>,<Ak,249>,<Ao,251>,<Ar,253>,<B0,304>,<B7,306>,<B8,308>,<CB,310>,<CF,313>,<CZ,327>,<Cg,333>,<Du,335>,<Cz,341>,<C8,349>,<C9,351>,<DA,353>,<DD,355>,<DE,358>,<DF,360>
D:<AJ,116>,<Do,99>,<AS,100>,<AV,101>,<Ab,35>,<Dq,104>,<Ag,105>,<Ak,105>,<Ao,106>,<Ar,107>,<A7,143>,<BV,150>,<Bb,152>,<Bv,160>,<Bx,161>,<B0,129>,<B7,130>,<CB,131>,<CF,132>,<CZ,138>,<Cb,175>,<Ce,176>,<Cg,141>,<Du,142>,<Cq,179>,<Cz,144>,<C8,148>,<C9,149>,<DA,150>,<DD,151>,<DE,151>,<DF,152>
E:<Dq,237>,<Ag,231>,<Ak,234>,<Ao,235>,<Ar,237>,<A3,351>,<Dr,354>,<A7,318>,<BR,370>,<BV,332>,<Bv,354>,<B0,285>,<Cg,312>,<Du,314>,<DD,333>,<DE,335>,<DF,337>
F:<Dn,1081>,<P,1263>,<Q,1193>,<AG,396>,<AJ,777>,<Do,816>,<Ab,294>,<Dp,1190>,<Ao,638>,<BM,1350>,<Bb,1248>,<CF,1085>,<CK,1560>,<CP,1570>,<CS,553>,<CT,556>,<Cz,1185>,<